### Nutzung der Helper-Klasse

In [1]:
from db_communication import db_communication
import pandas as pd
import numpy as np

In [2]:
config = {
    'user': 'root',
    'password': 'FDS-apm1',
    'host': 'min-ifm-xdm.ad.fh-bielefeld.de',
    'port': '3306',
    'database': 'NeueDB' 
}
my_db = db_communication(config)

Successfully connected.


In [3]:
tables = my_db.read_tables()
tables = [tables[i][0] for i in range(len(tables))]
tables

['Artikel', 'Auftrag', 'Auftragsposten', 'Einzelteile', 'Kunde', 'Stueckliste']

In [4]:
for table in tables:
    print(table)
    print(my_db.show_columns(table))

Artikel
[('ArtNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Bezeichnung', 'char(35)', 'NO', '', None, ''), ('Preis', 'decimal(7,2)', 'YES', '', None, ''), ('Montagezeit', 'int(11)', 'YES', '', None, ''), ('Ausstattung', 'char(35)', 'YES', '', None, '')]
Auftrag
[('AuftrNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Datum', 'date', 'YES', '', None, ''), ('KundenNr', 'int(11)', 'NO', 'MUL', None, '')]
Auftragsposten
[('PosNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('AuftragNr', 'int(11)', 'NO', 'MUL', None, ''), ('ArtikelNr', 'int(11)', 'NO', 'MUL', None, ''), ('Anzahl', 'int(11)', 'YES', '', None, ''), ('Gesamtpreis', 'decimal(7,2)', 'YES', '', None, '')]
Einzelteile
[('EinzelteilNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Bezeichnung', 'char(35)', 'NO', '', None, ''), ('Preis', 'decimal(7,2)', 'YES', '', None, '')]
Kunde
[('KundenNr', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('Name', 'char(30)', 'NO', '', None, ''), ('Vorname', 'char(30

### Funktionen nach Kategorie des zufälligen Wertes: 

In [5]:
import random
def randomElementInList(values):
    """
    Auswahl eines zufälligen Elements aus dem Wertebereich values
    Bsp.: ['Basic','Holland','BMX','Klassisch','Einrad'] ->  return = 'BMX'
    """
    return random.choice(values)

In [6]:
randomElementInList(['Basic','Holland','BMX','Klassisch','Einrad'])

'Holland'

In [7]:
def randomNumber(start,stop=None,step=1):
    """
    Generierung einer zufälligen Zahl nach random.randrange():
    Choose a random item from range(start, stop[, step])
    """
    return random.randrange(start=start,stop=stop,step=step)

In [8]:
randomNumber(1,20,1)

9

In [9]:
import time 
import datetime
def randomDate(start, end=datetime.datetime.now().strftime("%d.%m.%Y"), format="%d.%m.%Y", prop=random.random()):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

In [10]:
randomDate("12.10.2020","12.10.2022","%d.%m.%Y",random.random())

'12.06.2022'

In [11]:
def randomName():
    preNames = ["Tana","Francoise","Emmie","Stuart","Cedrick","Tiffani","Kym","Woodrow","Jolene","Autumn","Moriah","Claud","Antoinette",
                "Gisele","Chrissy","Ellena","Hilton","Susanne","Lindy","Nakesha","Rebecca","Lavenia","Donya","Claudia","Dreama","Alla",
                "Freeman","Weston","Jacki","Helene"]
    lastName = ["Allinder","Longley","Dobyns","Coldiron","Justin","Munroe","Boll","Clingerman","Pries","Santangelo","Miller","Bohman","Meachum",
                "Kamer","Waits","Polin","Suitt","Saulter","Pelt","Dapolito","Guild","Beeks","Hollingshead","Schissler","Cardamone","Sypher",
                "Fannin","Belizaire","Burleigh","Eslinger"]
    return randomElementInList(preNames), randomElementInList(lastName)

In [12]:
def randomMail(preName,lastName):
    providers = ['gmail.com','web.de','telekom.de','apple.com','fh-bielefeld.de']
    mail = "{0}.{1}@{2}".format(preName.lower(),lastName.lower(),randomElementInList(providers).lower())
    return mail

## Fill data

### Kundendaten

In [20]:
def fillCustomerTable(n,columnNamesDB=['Name','Vorname','Mail'],includeID=True):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Kunde'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Kunden-Tabelle in der DB (mit Ausnahme der KundenNr; Reihenfolge beachten!) 
    includeID - Ob KundenNr seperat mit angelegt werden soll
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    # Kundentabelle füllen
    table_name = 'Kunde'
    customerId = 'KundenNr'
    if includeID:
        customer_keys = np.concatenate(([customerId],columnNamesDB)) 
    else:
        customer_keys = columnNamesDB
    for i in range(n):
        customer_name = randomName()
        customer_values = i+1, customer_name[0], customer_name[1], randomMail(*customer_name)
        if includeID:
            customer_dict = dict(zip(customer_keys,customer_values))
        else:
            customer_dict = dict(zip(customer_keys,customer_values[1:]))
        my_db.insert_data(table_name,customer_dict)

In [21]:
fillCustomerTable(1,includeID=False)

INSERT INTO `Kunde` (`Name`, `Vorname`, `Mail`) VALUES ('Moriah', 'Waits', 'moriah.waits@apple.com');


### Aufträge

In [38]:
def fillOrderTable(n,columnNamesDB=['Datum','KundenNr'],includeID=True,
                   customerTableName='Kunde',customerId='KundenNr',startDate='2020-01-01'):
    """
    Hinzufügen von zufällig generierten Einträgen in der DB-Tabelle 'Aufträge'
    
    n - Anzahl der Einträge die hinzugefügt werden sollen
    columnNamesDB - Spaltennamen der Auftrag-Tabelle in der DB (mit Ausnahme der AuftrNr; Reihenfolge beachten!) 
    includeCustomerID - Ob AuftrNr seperat mit angelegt werden soll
    customerTableName - Name der DB-Tabelle 'Kunde'
    customerId - Name der Identifikationsnummer in customerTable
    startDate - Startdatum zur zufälligen Erzeugung
    
    Es werden die Inhalte nach den defaults in den Parametern erzeugt!
    Das Ändern der Reihenfolge der Spalten und die Hinzunahme weiterer Spalten muss im Quellcode angepasst werden!
    """
    table_name = 'Auftrag'
    orderId = 'AuftrNr'
    
    # Lesen der Wertebereiche aus relevanten Tabellen
    df_customers = my_db.get_table(customerTableName)
    
    # Auftragstabelle füllen
    if includeID:
        order_keys = np.concatenate(([customerId],columnNamesDB)) 
    else:
        order_keys = columnNamesDB
    for i in range(n):
        rand_orderDate = randomDate(start=startDate,
                                    end=datetime.datetime.now().strftime("%Y-%m-%d"),
                                    format="%Y-%m-%d",
                                    prop=random.random())
        rand_customerId = randomNumber(start=df_customers[customerId].astype(int).min(),
                                        stop=df_customers[customerId].astype(int).max())
        order_values = i+1, rand_orderDate, rand_customerId
        if includeID:
            order_dict = dict(zip(order_keys,order_values))
        else:
            order_dict = dict(zip(order_keys,order_values[1:]))
        my_db.insert_data(table_name,order_dict)

In [39]:
fillOrderTable(1)

INSERT INTO `Auftrag` (`KundenNr`, `Datum`) VALUES ('2', '2020-05-18');
